In [10]:
import numpy as np

In [33]:
def load_data(fpath):
    """
    Load input csv to np.array
    """
    return np.loadtxt(fpath, delimiter=',', dtype=np.int32)


def decode_input(data):
    """
    input: np.array with shape [:, 256]
    output:
        board         , np.array with shape (:, 8, 8)
        board_flipped , np.array with shape (:, 8, 8)
        player_plane  , np.array with shape (:, 8, 8) 
        target_move   , np.array with shape (:, 8, 8)
        
    """
    board = data.T[:64].T.reshape((len(data), 8, 8))
    board_flipped = data.T[64:128].T.reshape((len(data), 8, 8))
    player_plane = data.T[-128:-64].T.reshape((len(data), 8, 8))
    target_move = data.T[-64:].T.reshape((len(data), 8, 8))
    return board, board_flipped, player_plane, target_move


def encode_input(board, board_flipped, player_plane, target_move):
    """
    inputs:
        board         , np.array with shape (:, 8, 8)
        board_flipped , np.array with shape (:, 8, 8)
        player_plane  , np.array with shape (:, 8, 8)
        target_move   , np.array with shape (:, 8, 8)
    output:
        np.array with shape (:, 256)
    """
    record = []
    for board in (board, board_flipped, player_plane, target_move):
        board = board.flatten()
        record.extend(board)
    return record


def get_p(player_plane):
    """
    returns: True if row player
             False if col player
    """
    return player_plane[0][0] == 1
        

def get_move_plane_full(move_plane_short, player_plane):
    """
    Made to make symmetries work easier
    NB: 0 is row player and 1 is column player
    inputs: 
        move_plane_short
        [ [1, 0, 0],
          [0, 0, 0] ]
        player_plane
        [ [1, 1, 1],
          [1, 1, 1] ]
        
    output :
        move_plane_full
        [ [1, 1, 0],
          [0, 0, 0] ]
    """
    is_row_player = get_p(player_plane)
    move_plane_full = move_plane_short.copy()
    
    for i in range(len(move_plane_short)):
        for j in range(len(move_plane_short.T)):
            if move_plane_short[i][j] == 1:
                if is_row_player:
                    move_plane_full[i][j+1] = 1
                else:
                    move_plane_full[i+1][j] = 1
                return move_plane_full    
    return



def get_move_plane_short(move_plane_full):
    """
    made to make symmetries work easier
    inputs: 
        move_plane_full
        [ [1, 1, 0],
          [0, 0, 0] ]
    output:
        move_plane_short
        [ [1, 0, 0],
          [0, 0, 0] ]

    """
    move_full_flat = np.concatenate(move_plane_full).ravel()
    
    keep = True
    for i in range(len(move_full_flat)):
        if move_full_flat[i] and keep:
            keep = False
        elif move_full_flat[i]:
            move_plane_full_new = move_full_flat.copy()
            move_plane_full_new[i] = 0
            return move_plane_full_new
    return 

In [35]:
def symmetrics(board):
    """
    returns 4 symmetric versions of input board
    input, np.array with shape (8, 8)
    output 4*[np.array with shape (8, 8)]
    """
    syms = [
        board,
        np.rot90(board, k=2),
        np.flipud(board),
        np.fliplr(board) ]
    return syms


def rotate_symmetrics(board):
    """
    returns 4 symmetric versions of 90rotated input board
    input, np.array with shape (8, 8)
    output 4*[np.array with shape (8, 8)]
    """
    board = np.rot90(board, k=1)
    return symmetrics(board)


def flip_board(board):
    """
    flips 1s to 0s and vice-versa in np.array
    """
    return (~board.astype('bool_')).astype('int32')

In [42]:
# Load input data csv
import numpy as np
fpath = "/home/eolus/Desktop/Dauphine/metaheuristique/domineering.csv"

data = load_data(fpath)
print('Size of original dataset:', len(data))

board, board_flipped, player_plane, target_move = decode_input(data)

dataset_augmented_list = []
for row in zip(board, board_flipped, player_plane, target_move):
    
    # Values board, board_flipped, player_plane, target_move
    b   = row[0]
    b_f = row[1]
    p_p = row[2]
    t_m = row[3]
    
    # Symmetrics
    board_syms = symmetrics(b)
    board_flipped_syms = symmetrics(b_f)
    player_plane_syms = [p_p for _ in range(4)]
    target_move_syms = get_move_plane_short(symmetrics(get_move_plane_full(t_m, p_p)))
    
    syms = list(zip(board_syms, board_flipped_syms, player_plane_syms, target_move_syms))
    dataset_augmented_list.extend(syms)
    
    # Symmetrics rotated (for other player)
    board_rot_syms = rotate_symmetrics(b)
    board_flipped_rot_syms = rotate_symmetrics(b_f)
    player_plane_rot_syms = [flip_board(p_p) for _ in range(4)]
    target_move_rot_syms = get_move_plane_short(rotate_symmetrics(get_move_plane_full(t_m, p_p)))
    
    rotate_syms = list(zip(board_rot_syms, board_flipped_rot_syms, player_plane_rot_syms, target_move_rot_syms))
    dataset_augmented_list.extend(rotate_syms)

print("Size of augmented dataset:", len(dataset_augmented_list))
print("Sanity check x8:", "Passed" if len(data) * 8 == len(dataset_augmented_list) else "Failed")

Size of original dataset: 28155
Size of augmented dataset: 225240
Sanity check x8: Passed


*NB : How to treat duplicates X ?*

### Deep learning magic

In [17]:
from keras import backend as K
K.set_image_data_format('channels_first')

In [23]:
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()

model.add(Dense(12, input_dim=192, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(64, activation='sigmoid'))

model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

model.fit(X_train, y_train, epochs=5, batch_size=32)

loss_and_metrics = model.evaluate(X_test, y_test, batch_size=128)
loss_and_metrics